In [1]:
import pickle
objects = []
with (open("noun_phrase_results/entities_test", "rb")) as openfile:
    while True:
        try:
            objects.append(pickle.load(openfile))
        except EOFError:
            break
print len(objects[0])            
test = objects[0]
print len(test)
entities_test = test

6532
6532


In [2]:
import pickle
objects_train = []
with (open("noun_phrase_results/entities_train1", "rb")) as openfile:
    while True:
        try:
            objects_train.append(pickle.load(openfile))
        except EOFError:
            break
print len(objects_train[0])            
original_test = objects_train[0]
print len(original_test)
entities_train = original_test

11035
11035


In [3]:
# the original code is in the SentenceClassification.ipytn file. I am eliminating many variables in this file.
# This file is copied from the SentenceClassification_main file.
# In here we are trying to extract the data related to the value "Health" in the category Personal Information Type

import glob
import pandas as pd
import json
import csv
from bs4 import BeautifulSoup, NavigableString
import nltk.data

path = r'../Plain_Html'  # use your path
allFiles = glob.glob(path + "/*.html")
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

path = r'Train'  # use your path
allFiles = glob.glob(path + "/*.csv")


allFiles = allFiles[:65]


# all the followint positives are to be counted if the sentences are in that particular class. 
# A sen_neg to be counted in any other circumstances
sen0_pos = 0 # action_first_party no. of sentence positives
sen0_neg = 0 # action_first_party no. of sentence negatives
sen_actioFP = []
sen_actioFP_class = []

sen1_pos = 0 # Personal Information Type no. of sentence positives
sen1_neg = 0 # action_first_party no. of sentence negatives
sen_personalIT = []
sen_personalIT_class = []

sen2_pos = 0 # purpose no. of sentence negatives
sen2_neg = 0 # purpose no. of sentence negatives
sen_purpose = []
sen_purpose_class = []

sen_Others = []

firstParty = []
thirdParty = []
dataRetention = []
userControl = []

span_contact = []

sen3_pos = 0

def html_to_text(html):
    "Creates a formatted text email message as a string from a rendered html template (page)"
    html_report_part1 = open(html,'r')
    soup = BeautifulSoup(html_report_part1, 'html.parser')
    # Ignore anything in head tag
    text = []
    start_index = []
    end_index = []
    for element in soup.descendants:
        # We use type and not isinstance since comments, cdata, etc are subclasses that we don't want
        count = 0;
        if type(element) == NavigableString:
            # We use the assumption that other tags can't be inside a script or style
            if element.parent.name in ('script', 'style'):
                continue

            # remove any multiple and leading/trailing whitespace
            string = ' '.join(element.string.split())
            if string:
                if element.parent.name == 'a':
                    a_tag = element.parent
                    # replace link text with the link
                    string = a_tag['href']
                    # concatenate with any non-empty immediately previous string
                    if (    type(a_tag.previous_sibling) == NavigableString and
                            a_tag.previous_sibling.string.strip() ):
                        text[-1] = text[-1] + ' ' + string
                        continue
                elif element.previous_sibling and element.previous_sibling.name == 'a':
                    text[-1] = text[-1] + ' ' + string
                    continue
                elif element.parent.name == 'p':
                    # Add extra paragraph formatting newline
                    string = '\n' + string
                text += [string]
                start_index.append(count)
    doc = '\n'.join(text)
    return doc



for file in allFiles:
    
    df = pd.read_csv(file, thousands=',', header=None)
    len(df)
    number_of_segments = len(df) + 1

    file = file.split('Train/', 1)[1]

    ### beautifulsuop on html doc
    file_html = '../Plain_Html/'+file
    file_html = file_html.split('csv', 1)[0]
    file_html = file_html+'html'
    ls = []
    indeces = []
    data = html_to_text(file_html)
    ls = tokenizer.tokenize(data)
    count = 0
    for l in ls:
        indeces.append([count,count+len(l)])
        count = count+len(l)

    for i in range(number_of_segments-1):
        choice = 0 #whether the sentence is from one of the following categories
        if df[5][i] == "First Party Collection/Use":
            parse_json = json.loads(str(df[6][i]))            
            if parse_json["Personal Information Type"]["endIndexInSegment"] != -1 and parse_json["Personal Information Type"]["value"]=="User online activities":
                choice = 1
                str_index = parse_json["Personal Information Type"]["startIndexInSegment"]
                end_index = parse_json["Personal Information Type"]["endIndexInSegment"]
                span_contact.append(parse_json["Personal Information Type"]["selectedText"])
                for index in range(len(ls)):
                    parsed_sen = ls[index].replace("/\n|||-","")                        
                    parsed_sen = ls[index].replace("|||","") 
                    if((str_index > indeces[index][0] and end_index < indeces[index][1]) or 
                       (str_index < indeces[index][0] and end_index > indeces[index][1]) or
                       (str_index < indeces[index][0] and end_index > indeces[index][0] and end_index < indeces[index][1]) or
                       (str_index > indeces[index][0] and str_index < indeces[index][1] and end_index > indeces[index][1])):                        
                        sen_personalIT.append([parsed_sen,parse_json["Personal Information Type"]["selectedText"]])
                        firstParty.append([parsed_sen,parse_json["Personal Information Type"]["selectedText"]])

        if df[5][i] == "Third Party Sharing/Collection":
            parse_json = json.loads(str(df[6][i]))            
            if parse_json["Personal Information Type"]["endIndexInSegment"] != -1 and parse_json["Personal Information Type"]["value"]=="User online activities":
                choice = 1
                str_index = parse_json["Personal Information Type"]["startIndexInSegment"]
                end_index = parse_json["Personal Information Type"]["endIndexInSegment"]
                span_contact.append(parse_json["Personal Information Type"]["selectedText"])
                for index in range(len(ls)):
                    parsed_sen = ls[index].replace("/\n|||-","")                        
                    parsed_sen = ls[index].replace("|||","") 
                    if((str_index > indeces[index][0] and end_index < indeces[index][1]) or 
                       (str_index < indeces[index][0] and end_index > indeces[index][1]) or
                       (str_index < indeces[index][0] and end_index > indeces[index][0] and end_index < indeces[index][1]) or
                       (str_index > indeces[index][0] and str_index < indeces[index][1] and end_index > indeces[index][1])):                        
                        sen_personalIT.append([parsed_sen,parse_json["Personal Information Type"]["selectedText"]])
                        thirdParty.append([parsed_sen,parse_json["Personal Information Type"]["selectedText"]])
                        
        if df[5][i] == "User Choice/Control":
            parse_json = json.loads(str(df[6][i]))            
            if parse_json["Personal Information Type"]["endIndexInSegment"] != -1 and parse_json["Personal Information Type"]["value"]=="User online activities":
                choice = 1
                str_index = parse_json["Personal Information Type"]["startIndexInSegment"]
                end_index = parse_json["Personal Information Type"]["endIndexInSegment"]
                span_contact.append(parse_json["Personal Information Type"]["selectedText"])                
                for index in range(len(ls)):
                    parsed_sen = ls[index].replace("/\n|||-","")                        
                    parsed_sen = ls[index].replace("|||","") 
                    if((str_index > indeces[index][0] and end_index < indeces[index][1]) or 
                       (str_index < indeces[index][0] and end_index > indeces[index][1]) or
                       (str_index < indeces[index][0] and end_index > indeces[index][0] and end_index < indeces[index][1]) or
                       (str_index > indeces[index][0] and str_index < indeces[index][1] and end_index > indeces[index][1])):                        
                        sen_personalIT.append([parsed_sen,parse_json["Personal Information Type"]["selectedText"]])                        
                        userControl.append([parsed_sen,parse_json["Personal Information Type"]["selectedText"]])
                        
        if df[5][i] == "Data Retention":
            parse_json = json.loads(str(df[6][i]))            
            if parse_json["Personal Information Type"]["endIndexInSegment"] != -1 and parse_json["Personal Information Type"]["value"]=="User online activities":
                choice = 1
                str_index = parse_json["Personal Information Type"]["startIndexInSegment"]
                end_index = parse_json["Personal Information Type"]["endIndexInSegment"]
                span_contact.append(parse_json["Personal Information Type"]["selectedText"])                
                for index in range(len(ls)):
                    parsed_sen = ls[index].replace("/\n|||-","")                        
                    parsed_sen = ls[index].replace("|||","") 
                    if((str_index > indeces[index][0] and end_index < indeces[index][1]) or 
                       (str_index < indeces[index][0] and end_index > indeces[index][1]) or
                       (str_index < indeces[index][0] and end_index > indeces[index][0] and end_index < indeces[index][1]) or
                       (str_index > indeces[index][0] and str_index < indeces[index][1] and end_index > indeces[index][1])):                        
                        sen_personalIT.append([parsed_sen,parse_json["Personal Information Type"]["selectedText"]])
                        dataRetention.append([parsed_sen,parse_json["Personal Information Type"]["selectedText"]])
                        
print("len of the contact sentences from Personal Information Type")
print(len(sen_personalIT))

len of the contact sentences from Personal Information Type
962


In [4]:
# extracting the named entities form the annotated sets in the training dataset
from pycorenlp import StanfordCoreNLP
from nltk.tree import Tree

nlp = StanfordCoreNLP('http://localhost:9000')
train_list = []
train = []
for annot in sen_personalIT:
    output = nlp.annotate(str(annot[1]), properties={'annotators': 'tokenize,ssplit,pos,depparse,parse', 'outputFormat': 'json'})
#     print(output['sentences'][0]['parse'])
    parsestr=output['sentences'][0]['parse']
    for i in Tree.fromstring(parsestr).subtrees():
        if i.label() == 'NP':
            string = ""
            for leaf in i.leaves():
                if str(leaf)==',':
                    string += str(leaf)
                else:
                    string += " "+str(leaf)
#             print string.strip()
            train_list.append(string.strip())
                
train = list(set(train_list))
print len(train), len(train_list)
import pickle

with open('noun_phrase_results/entities_train_personalIT_location', 'wb') as fp:
    pickle.dump(train, fp)


1385 8390


In [5]:
from collections import Counter
import csv

with open('noun_phrase_results/health/entities_train_personalIT_location_csv.csv', 'wb') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    for i in Counter(train_list):
        wr.writerow([i])

In [6]:
Counter(train_list).most_common()

[('you', 639),
 ('information', 134),
 ('your visits', 108),
 ('the website', 99),
 ('pages', 92),
 ('our Website', 90),
 ('the Site', 89),
 ('we', 79),
 ('time', 70),
 ('the site', 70),
 ('use', 59),
 ('one', 53),
 ('the Sites', 53),
 ('our site', 52),
 ('URL', 49),
 ('your visit', 48),
 ('your', 46),
 ('data', 45),
 ('other sites', 45),
 ('page visits', 43),
 ('additional Other Information', 41),
 ('our features', 41),
 ('use of our features', 41),
 ("the Site 's", 41),
 ("the Site 's webpages", 41),
 ("your visits to the Site 's webpages", 41),
 ('activities', 40),
 ('responses', 40),
 ('purchasing history or preferences', 39),
 ('our Sites', 37),
 ('advertisements and promotions', 36),
 ('advertisements and promotions on the Sites', 36),
 ('our sites', 36),
 ('your visit to one of our Sites or apps', 36),
 ('sites and apps', 36),
 ('other sites following your visit to one of our Sites or apps', 36),
 ('our Sites or apps', 36),
 ('responses to advertisements and promotions on the Si

In [7]:


print len(set(train) & set(entities_train))
entities_train =  list(set(entities_train) - (set(train) & set(entities_train)))
print len(entities_train)



375
10660


In [8]:
import random
ls = random.sample(xrange(len(entities_train)), len(train))
print len(ls)

extracted_train = []
for i in ls:
    extracted_train.append(entities_train[i])

print len(extracted_train)

1385
1385


In [9]:
# the original code is in the SentenceClassification.ipytn file. I am eliminating many variables in this file.
# This file is copied from the SentenceClassification_main file.
# In here we are trying to extract the data related to the value "Health" in the category Personal Information Type

import glob
import pandas as pd
import json
import csv
from bs4 import BeautifulSoup, NavigableString
import nltk.data

path = r'Plain_Html'  # use your path
allFiles = glob.glob(path + "/*.html")
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

path = r'Test'  # use your path
allFiles = glob.glob(path + "/*.csv")


allFiles = allFiles[:65]

firstParty_test = []
thirdParty_test = []
dataRetention_test = []
userControl_test = []

sen_personalIT_test = []
sen_personalIT = []

def html_to_text(html):
    "Creates a formatted text email message as a string from a rendered html template (page)"
    html_report_part1 = open(html,'r')
    soup = BeautifulSoup(html_report_part1, 'html.parser')
    # Ignore anything in head tag
    text = []
    start_index = []
    end_index = []
    for element in soup.descendants:
        # We use type and not isinstance since comments, cdata, etc are subclasses that we don't want
        count = 0;
        if type(element) == NavigableString:
            # We use the assumption that other tags can't be inside a script or style
            if element.parent.name in ('script', 'style'):
                continue

            # remove any multiple and leading/trailing whitespace
            string = ' '.join(element.string.split())
            if string:
                if element.parent.name == 'a':
                    a_tag = element.parent
                    # replace link text with the link
                    string = a_tag['href']
                    # concatenate with any non-empty immediately previous string
                    if (    type(a_tag.previous_sibling) == NavigableString and
                            a_tag.previous_sibling.string.strip() ):
                        text[-1] = text[-1] + ' ' + string
                        continue
                elif element.previous_sibling and element.previous_sibling.name == 'a':
                    text[-1] = text[-1] + ' ' + string
                    continue
                elif element.parent.name == 'p':
                    # Add extra paragraph formatting newline
                    string = '\n' + string
                text += [string]
                start_index.append(count)
    doc = '\n'.join(text)
    return doc



for file in allFiles:
    
    df = pd.read_csv(file, thousands=',', header=None)
    len(df)
    number_of_segments = len(df) + 1

    file = file.split('Test/', 1)[1]

    ### beautifulsuop on html doc
    file_html = '../Plain_Html/'+file
    file_html = file_html.split('csv', 1)[0]
    file_html = file_html+'html'
    ls = []
    indeces = []
    data = html_to_text(file_html)
    ls = tokenizer.tokenize(data)
    count = 0
    for l in ls:
        indeces.append([count,count+len(l)])
        count = count+len(l)

    for i in range(number_of_segments-1):
        choice = 0 #whether the sentence is from one of the following categories
        if df[5][i] == "First Party Collection/Use":
            parse_json = json.loads(str(df[6][i]))            
            if parse_json["Personal Information Type"]["endIndexInSegment"] != -1 and parse_json["Personal Information Type"]["value"]=="User online activities":
                choice = 1
                str_index = parse_json["Personal Information Type"]["startIndexInSegment"]
                end_index = parse_json["Personal Information Type"]["endIndexInSegment"]
                
                for index in range(len(ls)):
                    parsed_sen = ls[index].replace("/\n|||-","")                        
                    parsed_sen = ls[index].replace("|||","") 
                    if((str_index > indeces[index][0] and end_index < indeces[index][1]) or 
                       (str_index < indeces[index][0] and end_index > indeces[index][1]) or
                       (str_index < indeces[index][0] and end_index > indeces[index][0] and end_index < indeces[index][1]) or
                       (str_index > indeces[index][0] and str_index < indeces[index][1] and end_index > indeces[index][1])):                        
                        sen_personalIT_test.append([parsed_sen,parse_json["Personal Information Type"]["selectedText"]])
                        firstParty_test.append([parsed_sen,parse_json["Personal Information Type"]["selectedText"]])
                        
        if df[5][i] == "Third Party Sharing/Collection":
            parse_json = json.loads(str(df[6][i]))            
            if parse_json["Personal Information Type"]["endIndexInSegment"] != -1 and parse_json["Personal Information Type"]["value"]=="User online activities":
                choice = 1
                str_index = parse_json["Personal Information Type"]["startIndexInSegment"]
                end_index = parse_json["Personal Information Type"]["endIndexInSegment"]
                
                for index in range(len(ls)):
                    parsed_sen = ls[index].replace("/\n|||-","")                        
                    parsed_sen = ls[index].replace("|||","") 
                    if((str_index > indeces[index][0] and end_index < indeces[index][1]) or 
                       (str_index < indeces[index][0] and end_index > indeces[index][1]) or
                       (str_index < indeces[index][0] and end_index > indeces[index][0] and end_index < indeces[index][1]) or
                       (str_index > indeces[index][0] and str_index < indeces[index][1] and end_index > indeces[index][1])):                        
                        sen_personalIT_test.append([parsed_sen,parse_json["Personal Information Type"]["selectedText"]])
                        thirdParty_test.append([parsed_sen,parse_json["Personal Information Type"]["selectedText"]])
                        
        if df[5][i] == "User Choice/Control":
            parse_json = json.loads(str(df[6][i]))            
            if parse_json["Personal Information Type"]["endIndexInSegment"] != -1 and parse_json["Personal Information Type"]["value"]=="User online activities":
                choice = 1
                str_index = parse_json["Personal Information Type"]["startIndexInSegment"]
                end_index = parse_json["Personal Information Type"]["endIndexInSegment"]
                
                for index in range(len(ls)):
                    parsed_sen = ls[index].replace("/\n|||-","")                        
                    parsed_sen = ls[index].replace("|||","") 
                    if((str_index > indeces[index][0] and end_index < indeces[index][1]) or 
                       (str_index < indeces[index][0] and end_index > indeces[index][1]) or
                       (str_index < indeces[index][0] and end_index > indeces[index][0] and end_index < indeces[index][1]) or
                       (str_index > indeces[index][0] and str_index < indeces[index][1] and end_index > indeces[index][1])):                        
                        sen_personalIT_test.append([parsed_sen,parse_json["Personal Information Type"]["selectedText"]])                        
                        userControl_test.append([parsed_sen,parse_json["Personal Information Type"]["selectedText"]])
        
        if df[5][i] == "Data Retention":
            parse_json = json.loads(str(df[6][i]))            
            if parse_json["Personal Information Type"]["endIndexInSegment"] != -1 and parse_json["Personal Information Type"]["value"]=="User online activities":
                choice = 1
                str_index = parse_json["Personal Information Type"]["startIndexInSegment"]
                end_index = parse_json["Personal Information Type"]["endIndexInSegment"]
                
                for index in range(len(ls)):
                    parsed_sen = ls[index].replace("/\n|||-","")                        
                    parsed_sen = ls[index].replace("|||","") 
                    if((str_index > indeces[index][0] and end_index < indeces[index][1]) or 
                       (str_index < indeces[index][0] and end_index > indeces[index][1]) or
                       (str_index < indeces[index][0] and end_index > indeces[index][0] and end_index < indeces[index][1]) or
                       (str_index > indeces[index][0] and str_index < indeces[index][1] and end_index > indeces[index][1])):                        
                        sen_personalIT_test.append([parsed_sen,parse_json["Personal Information Type"]["selectedText"]])
                        dataRetention_test.append([parsed_sen,parse_json["Personal Information Type"]["selectedText"]])
                        
print("len of the contact sentences from Personal Information Type for test data")
print len(sen_personalIT_test)

len of the contact sentences from Personal Information Type for test data
379


In [10]:
test_pos1 = []
for annot in sen_personalIT_test:
    output = nlp.annotate(str(annot[1]), properties={'annotators': 'tokenize,ssplit,pos,depparse,parse', 'outputFormat': 'json'})
#     print(output['sentences'][0]['parse'])
    parsestr=output['sentences'][0]['parse']
    for i in Tree.fromstring(parsestr).subtrees():
        if i.label() == 'NP':
            string = ""
            for leaf in i.leaves():
                if str(leaf)==',':
                    string += str(leaf)
                else:
                    string += " "+str(leaf)
            
            test_pos1.append(string.strip())
                
test_pos1 = list(set(test_pos1))
print len(test_pos1)
with open('noun_phrase_results/health/entities_train_personalIT_health_test', 'wb') as fp:
    pickle.dump(train, fp)


772


In [11]:
print len(set(test_pos1) & set(entities_test))
entities_test =  list(set(entities_test) - (set(test_pos1) & set(entities_test)))
print len(entities_test)

349
6183


In [12]:
print entities_test[:10]
print test_pos1[:10]

['these companies', 'other facility locations', 'third-party sites and applications', 'alert you of new products or services, features, or enhancements ; handle/route your customer service questions or issues', 'Track Signals', 'small bits', 'online interactive communities, your Devices, Digital Content, Apps and software, and display associated content and advertising', "a child 's first name and the friend 's email address", 'service bulletins', 'a safe online experience']
['the search engine you used to find the Curse Features', 'the categories', 'other click-stream data', 'entry and exit points', 'activity on USA.gov', 'results', 'the items', 'your Service usage activities', 'use of the Sites .', 'CPU']


In [13]:
## Naive Bayesian
t_set = []
for t in train:
    t_set.append((unicode(t, 'utf-8'),'positive'))

for t in extracted_train:
    t_set.append((unicode(t, 'utf-8'),'negative'))
    
print len(t_set)

train_set = []
for (words, sentiment) in t_set:
    words_filtered = [e.lower() for e in words.split() if len(e) >= 3]
    train_set.append((words_filtered, sentiment))

print train_set[:10]

2770
[([u'your', u'interactions', u'with', u'the', u'websites,', u'apps,', u'and', u'other', u'services', u'you', u'use,', u'the', u'content', u'you', u'view,', u'the', u'search', u'queries', u'you', u'submit,'], 'positive'), ([u'user', u'name,', u'address,', u'other', u'personal', u'information'], 'positive'), ([u'clickstream', u'to,', u'and', u'from', u'our', u'services,', u'including', u'date', u'and', u'time', u'and', u'products', u'you', u'viewed', u'searched'], 'positive'), ([u'other', u'sites', u'subscribers'], 'positive'), ([u'your', u'purchases'], 'positive'), ([u'your', u'device'], 'positive'), ([u'information', u'about', u'user', u'visits'], 'positive'), ([u'pages', u'visited'], 'positive'), ([u'service', u'-lrb-', u'-rrb-', u'you', u'arrived', u'from,', u'and', u'other', u'clickstream', u'data'], 'positive'), ([u'description', u'the', u'page'], 'positive')]


In [14]:
# testing data set prep

t_set_test = []
for t in test_pos1[:200]:
    t_set_test.append((unicode(t, 'utf-8'),'positive'))

for t in entities_test[:200]:
    t_set_test.append((unicode(t, 'utf-8'),'negative'))
    
print len(t_set_test)

test_set = []
for (words, sentiment) in t_set_test:
    words_filtered = [e.lower() for e in words.split() if len(e) >= 3]
    test_set.append((words_filtered, sentiment))

400


In [16]:
from textblob.classifiers import NaiveBayesClassifier
cl1 = NaiveBayesClassifier(t_set)

In [17]:
count_pos1 = 0
count_neg1 = 0
for t in range(len(test_pos1[:20])):
    value = cl1.classify(test_pos1[t])
    if value is "negative": 
        count_neg1 = count_neg1+1
    if value is "positive":
        count_pos1 = count_pos1+1
print count_pos1, count_neg1

20 0


In [18]:
count_pos2 = 0
count_neg2 = 0
for t in range(len(entities_test[:20])):
    value = cl1.classify(entities_test[t])
    if value is "negative":
        count_neg2 = count_neg2+1
    if value is "positive":
        count_pos2 = count_pos2+1
print count_pos2, count_neg2

12 8


In [19]:
print "Naive Bayesian"
print "true positives  -", count_pos1
print "false negatives  -", count_neg1
print "false positives -", count_pos2
print "true negatives -", count_neg2
numerator = count_pos1 + count_neg2
denominator = count_pos1 + count_neg2 + count_pos2 + count_neg1
accuracy = numerator * 100 / denominator
print "accuracy  - ", accuracy
print "recall    - ", 2000/20 # TP/(TP+FN)
print "precision - ", 2000/32

Naive Bayesian
true positives  - 20
false negatives  - 0
false positives - 12
true negatives - 8
accuracy  -  70
recall    -  100
precision -  62


count_pos1 = 0
count_neg1 = 0
for t in range(len(test_pos1[:50])):
    value = classifier1.classify(test_pos1[t])
    if value is False:
        count_neg1 = count_neg1+1
    if value is True:
        count_pos1 = count_pos1+1
print count_pos1, count_neg1

In [15]:
naive_bayesian_train_set = []
naive_bayesian_train_set.extend(train)
naive_bayesian_train_set.extend(extracted_train)
print len(naive_bayesian_train_set)

naive_bayesian_train_label_set = []
for t in train:
    naive_bayesian_train_label_set.append('positive')

for t in extracted_train:
    naive_bayesian_train_label_set.append('negative')
    
print len(naive_bayesian_train_label_set)

2770
2770


In [17]:
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

pipeline = Pipeline([
    ('vectorizer',  CountVectorizer()),
    ('tfidf_transformer',  TfidfTransformer()),
    ('classifier',  MultinomialNB()) ])



pipeline.fit(naive_bayesian_train_set, naive_bayesian_train_label_set)
# pipeline.predict(examples) # ['spam', 'ham']

Pipeline(steps=[('vectorizer', CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
    ...         use_idf=True)), ('classifier', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [21]:
naive_bayesian_train_set_test = []
for t in test_pos1[:200]:
    naive_bayesian_train_set_test.append(t)

for t in entities_test[:200]:
    naive_bayesian_train_set_test.append(t)
    
naive_bayesian_train_label_set_test = []
for t in test_pos1[:200]:
    naive_bayesian_train_label_set_test.append('positive')

for t in entities_test[:200]:
    naive_bayesian_train_label_set_test.append('negative')

    

In [22]:

from sklearn.metrics import classification_report
predicted_values = pipeline.predict(naive_bayesian_train_set_test) # ['spam', 'ham']
print(classification_report(naive_bayesian_train_label_set_test, predicted_values))

print len(predicted_values)

             precision    recall  f1-score   support

   negative       0.76      0.68      0.72       200
   positive       0.71      0.79      0.75       200

avg / total       0.74      0.73      0.73       400

400


In [23]:
# SVM
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
from sklearn.feature_extraction.text import TfidfTransformer


scores = ['precision', 'recall','accuracy']

pipeline = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, n_iter=5, random_state=42))])


_ = pipeline.fit(naive_bayesian_train_set, naive_bayesian_train_label_set)
svm_train_label_set_test_predict = pipeline.predict(naive_bayesian_train_set_test)



In [24]:
from sklearn.metrics import classification_report

print(classification_report(naive_bayesian_train_label_set_test, svm_train_label_set_test_predict))

             precision    recall  f1-score   support

   negative       0.70      0.79      0.74       200
   positive       0.76      0.66      0.71       200

avg / total       0.73      0.72      0.72       400



In [60]:
print MultinomialNB().get_params()
print svm.OneClassSVM().get_params()

{'alpha': 1.0, 'fit_prior': True, 'class_prior': None}
{'kernel': 'rbf', 'verbose': False, 'degree': 3, 'shrinking': True, 'max_iter': -1, 'random_state': None, 'tol': 0.001, 'cache_size': 200, 'coef0': 0.0, 'nu': 0.5, 'gamma': 'auto'}


In [68]:
C_range = np.logspace(-2, 10, 13)
gamma_range = np.logspace(-3, -1, 1000)
alpha_range = np.logspace(-3, -1, 1000)

param_grid = dict(clf__alpha=gamma_range)
print alpha_range

[ 0.001       0.00100462  0.00100926  0.00101393  0.00101861  0.00102332
  0.00102804  0.00103279  0.00103757  0.00104236  0.00104718  0.00105202
  0.00105688  0.00106176  0.00106666  0.00107159  0.00107654  0.00108152
  0.00108652  0.00109154  0.00109658  0.00110165  0.00110674  0.00111185
  0.00111699  0.00112215  0.00112733  0.00113254  0.00113777  0.00114303
  0.00114831  0.00115362  0.00115895  0.0011643   0.00116968  0.00117509
  0.00118052  0.00118597  0.00119145  0.00119696  0.00120249  0.00120804
  0.00121362  0.00121923  0.00122486  0.00123052  0.00123621  0.00124192
  0.00124766  0.00125342  0.00125922  0.00126503  0.00127088  0.00127675
  0.00128265  0.00128858  0.00129453  0.00130051  0.00130652  0.00131256
  0.00131862  0.00132471  0.00133083  0.00133698  0.00134316  0.00134937
  0.0013556   0.00136187  0.00136816  0.00137448  0.00138083  0.00138721
  0.00139362  0.00140006  0.00140653  0.00141303  0.00141955  0.00142611
  0.0014327   0.00143932  0.00144597  0.00145265  0

In [87]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.cross_validation import KFold

scores = ['precision', 'recall','accuracy']

pipeline = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB())])

# cv = StratifiedShuffleSplit(n_splits=10, test_size=0.2, random_state=42)


grid = GridSearchCV(pipeline, param_grid=param_grid, scoring="accuracy", cv=10)
grid.fit(naive_bayesian_train_set, naive_bayesian_train_label_set)


# print "Performing grid search..."
# print "pipeline:", [name for name, _ in pipeline.steps]
# print "parameters:"
# # pprint parameters
# grid_search.fit(train, data_targets)
# print ""

# print("Best score: %0.3f" % grid_search.best_score_)
# print("Best parameters set:")
# best_parameters = grid_search.best_estimator_.get_params()
# for param_name in sorted(parameters.keys()):
#     print("\t%s: %r" % (param_name, best_parameters[param_name]))

GridSearchCV(cv=10, error_score='raise',
       estimator=Pipeline(steps=[('vect', CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        st...False,
         use_idf=True)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))]),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'clf__alpha': array([ 0.001  ,  0.001  , ...,  0.09954,  0.1    ])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='accuracy', verbose=0)

In [91]:
from sklearn.metrics import classification_report
# no. of folds  = 10
print grid.best_estimator_.get_params()['clf'].get_params()['alpha'] 
print("Best score: %0.3f" % grid.best_score_)
print "predicted accuracy ", grid.score(naive_bayesian_train_set_test, naive_bayesian_train_label_set_test) 

naive_bayesian_train_label_set_test_predict = grid.predict(naive_bayesian_train_set_test)
print(classification_report(naive_bayesian_train_label_set_test, naive_bayesian_train_label_set_test_predict))

0.001
Best score: 0.887
predicted accuracy  0.825
             precision    recall  f1-score   support

   negative       0.80      0.87      0.83       100
   positive       0.86      0.78      0.82       100

avg / total       0.83      0.82      0.82       200



In [115]:
C = range(1, 10)
param_grid_svm = [
  {'clf__C': C, 'clf__kernel': ['linear']},
  {'clf__C': C, 'clf__gamma': np.logspace(-3, -1, 100), 'clf__kernel': ['rbf']},
]
print svm.SVC().get_params().keys()

['kernel', 'C', 'verbose', 'probability', 'degree', 'shrinking', 'max_iter', 'decision_function_shape', 'random_state', 'tol', 'cache_size', 'coef0', 'gamma', 'class_weight']


In [116]:
# SVM
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.pipeline import Pipeline
from sklearn import svm
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.cross_validation import KFold

scores = ['precision', 'recall','accuracy']

pipeline = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', svm.SVC())])

# cv = StratifiedShuffleSplit(n_splits=10, test_size=0.2, random_state=42)


grid = GridSearchCV(pipeline, param_grid=param_grid_svm, scoring="accuracy", cv=10)
grid.fit(naive_bayesian_train_set, naive_bayesian_train_label_set)


# print "Performing grid search..."
# print "pipeline:", [name for name, _ in pipeline.steps]
# print "parameters:"
# # pprint parameters
# grid_search.fit(train, data_targets)
# print ""

# print("Best score: %0.3f" % grid_search.best_score_)
# print("Best parameters set:")
# best_parameters = grid_search.best_estimator_.get_params()
# for param_name in sorted(parameters.keys()):
#     print("\t%s: %r" % (param_name, best_parameters[param_name]))

GridSearchCV(cv=10, error_score='raise',
       estimator=Pipeline(steps=[('vect', CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        st...,
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False))]),
       fit_params={}, iid=True, n_jobs=1,
       param_grid=[{'clf__C': [1, 2, 3, 4, 5, 6, 7, 8, 9], 'clf__kernel': ['linear']}, {'clf__gamma': array([ 0.001  ,  0.00105, ...,  0.09545,  0.1    ]), 'clf__C': [1, 2, 3, 4, 5, 6, 7, 8, 9], 'clf__kernel': ['rbf']}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='accuracy', verbose=0)

In [133]:
from sklearn.metrics import classification_report
print 'Best C:',grid.best_estimator_.get_params()['clf'].get_params()['C']
print 'Best Kernel:',grid.best_estimator_.get_params()['clf__kernel']
print 'Best Gamma:',grid.best_estimator_.get_params()['clf'].get_params()['gamma']
print "Best score: %0.3f" % grid.best_score_
print "predicted accuracy ", grid.score(naive_bayesian_train_set_test, naive_bayesian_train_label_set_test) 

naive_bayesian_train_label_set_test_predict = grid.predict(naive_bayesian_train_set_test)
print(classification_report(naive_bayesian_train_label_set_test, naive_bayesian_train_label_set_test_predict))

Best C: 5
Best Kernel: rbf
Best Gamma: 0.0911162756115
Best score: 0.876
predicted accuracy  0.865
             precision    recall  f1-score   support

   negative       0.85      0.89      0.87       100
   positive       0.88      0.84      0.86       100

avg / total       0.87      0.86      0.86       200



In [141]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedShuffleSplit


scores = ['precision', 'recall']

pipeline = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', svm.OneClassSVM())])

nu_range = [0.1, 0.3, 0.5, 0.7]
gamma_range = np.logspace(-9, 3, 13)
param_grid = dict(clf__nu = nu_range, clf__gamma=gamma_range)
# cv = StratifiedShuffleSplit(n_splits=10, test_size=0.2, random_state=42)

grid_oneclass = GridSearchCV(pipeline, param_grid=param_grid, scoring="accuracy")
grid_oneclass.fit(naive_bayesian_train_set, naive_bayesian_train_label_set)
print("The best parameters are %s with a score of %0.3f"
      % (grid_oneclass.best_params_, grid_oneclass.best_score_))

# print "Performing grid search..."
# print "pipeline:", [name for name, _ in pipeline.steps]
# print "parameters:"
# # pprint parameters
# grid_search.fit(train, data_targets)
# print ""

# print("Best score: %0.3f" % grid_search.best_score_)
# print("Best parameters set:")
# best_parameters = grid_search.best_estimator_.get_params()
# for param_name in sorted(parameters.keys()):
#     print("\t%s: %r" % (param_name, best_parameters[param_name]))

The best parameters are {'clf__gamma': 1.0000000000000001e-09, 'clf__nu': 0.1} with a score of 0.000


In [163]:
from sklearn.metrics import classification_report
# print 'Best C:',grid_oneclass.best_estimator_.get_params()['clf'].get_params()['C']
# print 'Best Kernel:',grid_oneclass.best_estimator_.get_params()['clf__kernel']
# print 'Best Gamma:',grid_oneclass.best_estimator_.get_params()['clf'].get_params()['gamma']
# print "Best score: %0.3f" % grid_oneclass.best_score_
print "predicted accuracy ", grid_oneclass.score(naive_bayesian_train_set_test, naive_bayesian_train_label_set_test) 

naive_bayesian_train_label_set_test_predict = grid_oneclass.predict(naive_bayesian_train_set_test)
print(classification_report(naive_bayesian_train_label_set_test, naive_bayesian_train_label_set_test_predict))

predicted accuracy  0.0


ValueError: Mix of label input types (string and number)

In [160]:
firstParty_entities = []
firstParty_labels = []
for i in firstParty_train:
    firstParty_labels.append('positive')
    firstParty_entities.append(i)
    
for i in range(len(firstParty_train)):
    firstParty_labels.append('negative')
    firstParty_entities.append(entities_train[i])
    

In [161]:
#### Combination Analysis
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.cross_validation import KFold

scores = ['precision', 'recall','accuracy']

pipeline = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB())])

# cv = StratifiedShuffleSplit(n_splits=10, test_size=0.2, random_state=42)
C_range = np.logspace(-2, 10, 5)
gamma_range = np.logspace(-3, -1, 10)
alpha_range = np.logspace(-3, -1, 10)

param_grid = dict(clf__alpha=gamma_range)

grid_firstParty = GridSearchCV(pipeline, param_grid=param_grid, scoring="accuracy")
grid_firstParty.fit(firstParty_entities, firstParty_labels)


# print "Performing grid search..."
# print "pipeline:", [name for name, _ in pipeline.steps]
# print "parameters:"
# # pprint parameters
# grid_search.fit(train, data_targets)
# print ""

# print("Best score: %0.3f" % grid_search.best_score_)
# print("Best parameters set:")
# best_parameters = grid_search.best_estimator_.get_params()
# for param_name in sorted(parameters.keys()):
#     print("\t%s: %r" % (param_name, best_parameters[param_name]))




GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(steps=[('vect', CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        st...False,
         use_idf=True)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))]),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'clf__alpha': array([ 0.001  ,  0.00167,  0.00278,  0.00464,  0.00774,  0.01292,
        0.02154,  0.03594,  0.05995,  0.1    ])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='accuracy', verbose=0)

In [162]:
from sklearn.metrics import classification_report
# print 'Best C:',grid_oneclass.best_estimator_.get_params()['clf'].get_params()['C']
# print 'Best Kernel:',grid_oneclass.best_estimator_.get_params()['clf__kernel']
# print 'Best Gamma:',grid_oneclass.best_estimator_.get_params()['clf'].get_params()['gamma']
# print "Best score: %0.3f" % grid_oneclass.best_score_
print "predicted accuracy ", grid_firstParty.score(naive_bayesian_train_set_test, naive_bayesian_train_label_set_test) 

naive_bayesian_train_label_set_test_predict = grid_firstParty.predict(naive_bayesian_train_set_test)
print(classification_report(naive_bayesian_train_label_set_test, naive_bayesian_train_label_set_test_predict))

predicted accuracy  0.82
             precision    recall  f1-score   support

   negative       0.78      0.89      0.83       100
   positive       0.87      0.75      0.81       100

avg / total       0.83      0.82      0.82       200

